In this pre-processing project we will be cleaning up and enhancing the dataframe to get it prepared for its use in the web app, then writing the code for streamlit.

In [53]:
#import packages
import pandas as pd
import numpy as np
import plotly.express as px
import streamlit as st
import altair as alt

In [54]:
# read CSV
vehicles_df = pd.read_csv('C:/Users/Tyler Wilson/Desktop/Sprint-4-Project/vehicles_us.csv')

In [55]:
# overviewing data
vehicles_df.info()
vehicles_df.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
2491,22995,2012.0,jeep wrangler unlimited,good,6.0,gas,153680.0,automatic,SUV,silver,1.0,2018-11-25,42
21400,8500,NaN,toyota corolla,excellent,4.0,gas,70200.0,automatic,other,black,NaN,2018-12-20,35
24581,16199,2010.0,chevrolet tahoe,like new,8.0,gas,104800.0,automatic,SUV,grey,1.0,2019-02-11,52
42280,7000,NaN,chevrolet silverado,like new,8.0,gas,180000.0,automatic,truck,red,1.0,2018-05-24,16
26137,9995,2003.0,ford f-150,excellent,8.0,gas,137302.0,automatic,truck,white,1.0,2018-06-23,83
31665,4899,2004.0,ford focus se,excellent,4.0,gas,88472.0,automatic,sedan,black,NaN,2018-07-15,37
30761,16995,2011.0,toyota rav4,excellent,4.0,gas,66000.0,automatic,SUV,silver,1.0,2018-10-23,42
11105,5900,2014.0,chrysler 200,excellent,4.0,gas,148272.0,automatic,sedan,red,NaN,2018-05-30,68
27377,25900,2015.0,toyota tacoma,good,6.0,gas,74379.0,automatic,truck,silver,1.0,2018-10-03,36
25778,8995,2013.0,chevrolet camaro,good,6.0,gas,179366.0,automatic,coupe,NaN,NaN,2018-12-10,12


In [56]:
# check for duplicate rows
print(vehicles_df.duplicated().sum())

0


There are a lot of missing values in the dataframe. Let's take care of those.

In [57]:
# assuming that the NaN values in the 'is_4wd' column mean that the vehicle is not a 4-wheel drive, let's replace them with a 0.
vehicles_df['is_4wd'] = vehicles_df['is_4wd'].fillna(0)


In [58]:
# calculate median model year for each model
model_medians = vehicles_df.groupby('model')['model_year'].median()

# function to fill missing model_year
def fill_model_year(row):
    if np.isnan(row['model_year']):
        return model_medians[row['model']]
    return row['model_year']

# apply the function to fill missing values
vehicles_df['model_year'] = vehicles_df.apply(fill_model_year, axis=1)

In [59]:
# calculate median cylinder count for each model
cylinder_medians = vehicles_df.groupby('model')['cylinders'].median()

# function to fill missing cylinders
def fill_cylinders(row):
    if np.isnan(row['cylinders']):
        return cylinder_medians[row['model']]
    return row['cylinders']

# apply the function to fill missing values
vehicles_df['cylinders'] = vehicles_df.apply(fill_cylinders, axis=1)

In [60]:
# calculate median cylinder count for each model
odometer_medians = vehicles_df.groupby('model')['odometer'].median()

# function to fill missing cylinders
def fill_odometer(row):
    if np.isnan(row['odometer']):
        return odometer_medians[row['model']]
    return row['odometer']

# apply the function to fill missing values
vehicles_df['odometer'] = vehicles_df.apply(fill_odometer, axis=1)

In [61]:
# drop the rest of the rows with missing values
vehicles_df = vehicles_df.dropna().reset_index(drop=True)

Let's add enhancements to the dataframe to make it easier to work with and clean it up before perparing the web app.

In [62]:
# convert float64 columns to int64
vehicles_df['model_year'] = vehicles_df['model_year'].astype('int64')
vehicles_df['cylinders'] = vehicles_df['cylinders'].astype('int64')
vehicles_df['odometer'] = vehicles_df['odometer'].astype('int64')
vehicles_df['is_4wd'] = vehicles_df['is_4wd'].astype('int64')

# convert 'date_posted' column to datetime
vehicles_df['date_posted'] = pd.to_datetime(vehicles_df['date_posted'])


In [63]:
# calculate IQR for 'model_year' and 'price', then define boundaries
Q1_model_year = vehicles_df['model_year'].quantile(0.25)
Q3_model_year = vehicles_df['model_year'].quantile(0.75)
IQR_model_year = Q3_model_year - Q1_model_year

lower_bound_model_year = Q1_model_year - 1.5 * IQR_model_year
upper_bound_model_year = Q3_model_year + 1.5 * IQR_model_year

Q1_price = vehicles_df['price'].quantile(0.25)
Q3_price = vehicles_df['price'].quantile(0.75)
IQR_price = Q3_price - Q1_price

lower_bound_price = Q1_price - 1.5 * IQR_price
upper_bound_price = Q3_price + 1.5 * IQR_price

# filter outliers
vehicles_df_filtered = vehicles_df[
    (vehicles_df['model_year'] >= lower_bound_model_year) &
    (vehicles_df['model_year'] <= upper_bound_model_year) &
    (vehicles_df['price'] >= lower_bound_price) &
    (vehicles_df['price'] <= upper_bound_price)
]

In [64]:
vehicles_df.to_csv('cleaned_vehicles.csv')

Now that the dataframe is cleaned, let's write the code for the web app.

In [65]:
# set title
st.title('Vehicle Data Analysis')

# make scatter plot
st.header('Price vs Odometer Value')

# set checkbox
show_condition_color = st.checkbox('Color by Condition')

if show_condition_color:
    scatter_fig = px.scatter(vehicles_df, x='odometer', y='price', color='condition',
                             title='Price vs Odometer Value (Colored by Condition)',
                             labels={'odometer': 'Odometer (miles)', 'price': 'Price (USD)', 'condition': 'Condition'})
else:
    scatter_fig = px.scatter(vehicles_df, x='odometer', y='price',
                             title='Price vs Odometer Value',
                             labels={'odometer': 'Odometer (miles)', 'price': 'Price (USD)'})

st.plotly_chart(scatter_fig)

# make histogram
st.header('Distribution of Vehicle Conditions')

hist_fig = px.histogram(vehicles_df, x='condition', title='Vehicle Condition Distribution',
                        labels={'condition': 'Condition', 'count': 'Count'})

st.plotly_chart(hist_fig)

DeltaGenerator()